In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

2023-05-17 13:45:38.224541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 13:45:39.178935: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-17 13:45:39.179087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

In [3]:
# check available GPUs
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


2023-05-17 13:45:42.238246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 13:45:42.251447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-17 13:45:42.253106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
# read file data.npz
dataset = np.load('data2.npz', allow_pickle=True)['data2'][()]


In [5]:
# create a dictionary to map the class names to integer labels
label_dict = {key: v for v, key in enumerate(dataset.keys())}
print(label_dict)

{'bed': 0, 'cat': 1, 'dog': 2, 'five': 3, 'happy': 4, 'left': 5, 'marvin': 6, 'sheila': 7, 'six': 8, 'stop': 9}


### Create dataset

In [6]:
data_dict = {
    'lmfcc': [],
    'mspec': [],
    'label': []
}
for key in dataset.keys():
    for utt in dataset[key]:
        if utt['lmfcc'].shape[1] != 44:
                    utt['lmfcc'] = np.pad(utt['lmfcc'], ((0,0),(0,44-utt['lmfcc'].shape[1])), 'constant', constant_values=0)
        if utt['mspec'].shape[1] != 128:
                    utt['mspec'] = np.pad(utt['mspec'], ((0,0),(0,128-utt['mspec'].shape[1])), 'constant', constant_values=0)
        data_dict['lmfcc'].append(utt['lmfcc'])
        data_dict['mspec'].append(utt['mspec'])
        data_dict['label'].append(label_dict[key])

# convert to numpy array
data_dict['lmfcc'] = np.array(data_dict['lmfcc'])
data_dict['mspec'] = np.array(data_dict['mspec'])
data_dict['label'] = np.array(data_dict['label'])

data = data_dict
print(data['lmfcc'].shape, data['mspec'].shape, data['label'].shape)

(19873, 13, 44) (19873, 128, 128) (19873,)


In [7]:
# expand the dimension of lmfcc to (N, T, D, 1) for CNN
data['lmfcc'] = np.expand_dims(data['lmfcc'], axis=-1)
data['mspec'] = np.expand_dims(data['mspec'], axis=-1)

### Split dataset into train, validation and test sets

In [8]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# split the dataset into train, validation and test sets
train_data, test_data, train_label, test_label = train_test_split(data['lmfcc'], data['label'], test_size=0.2, random_state=42)
train_data, val_data, train_label, val_label = train_test_split(train_data, train_label, test_size=0.2, random_state=42)

print(train_data.shape, train_label.shape)
print(val_data.shape, val_label.shape)
print(test_data.shape, test_label.shape)

# Convert labels to one-hot vectors
train_label = to_categorical(train_label)
val_label = to_categorical(val_label)
test_label = to_categorical(test_label)


(12718, 13, 44, 1) (12718,)
(3180, 13, 44, 1) (3180,)
(3975, 13, 44, 1) (3975,)


In [9]:
import itertools
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.regularizers import l2

mfcc_height = data['lmfcc'].shape[1]
mfcc_width = data['lmfcc'].shape[2]
num_classes = len(dataset.keys())

### Grid search for model architecture

In [10]:
# num_layers_list = [3, 4]
# num_filters_list = [64, 128]
# optimizer_list = [SGD(), Adam()]
# num_nodes_list = [64, 128]
# batch_size_list = [32, 64, 128]
# regularization_list = [Dropout(0.2), Dropout(0.3), None, BatchNormalization()]

# with open('results.txt', 'a') as file:
#     for num_layers, num_filters, optimizer, num_nodes, batch_size, regularization in itertools.product(num_layers_list, num_filters_list, optimizer_list, num_nodes_list, batch_size_list, regularization_list):
#         try:
#             model = Sequential()

#             model.add(Conv2D(num_filters, (3,3), activation='relu', input_shape=(mfcc_height, mfcc_width, 1)))

#             # Add convolutional layers
#             for _ in range(num_layers):
#                 model.add(Conv2D(num_filters, kernel_size=(3, 3), activation='relu', padding='same'))
#                 model.add(MaxPooling2D(pool_size=(2, 2)))
#                 if regularization is not None:
#                     model.add(regularization)

#             model.add(Flatten())

#             # Add fully connected layers
#             for _ in range(num_layers):
#                 model.add(Dense(num_nodes, activation='relu'))
#                 if regularization is not None:
#                     model.add(regularization)

#             model.add(Dense(10, activation='softmax'))  # Assuming a classification task with 10 classes

#             # Compile the model
#             model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#             history = model.fit(train_data, train_label,
#                                 validation_data=(val_data, val_label),
#                                 epochs=20,
#                                 batch_size=batch_size
#                                 )
            
#             val_accuracy = history.history['val_accuracy'][-1]  # Get the validation accuracy from the last epoch

#             print(f"Validation Accuracy: {val_accuracy}")

#             file.write(f"Model: num_layers={num_layers}, num_filters={num_filters}, optimizer={optimizer.__class__.__name__}, num_nodes={num_nodes}, batch_size={batch_size}, regularization={regularization}\n")
#             file.write(f"Validation Accuracy: {val_accuracy}\n\n")
#         except:
#             pass


In [16]:
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', input_shape=(mfcc_height, mfcc_width, 1)))
model.add(MaxPooling2D((3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), activation='relu', input_shape=(mfcc_height, mfcc_width, 1)))
model.add(MaxPooling2D((3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(256, (2,2), activation='relu', input_shape=(mfcc_height, mfcc_width, 1)))
model.add(MaxPooling2D((3,3), strides=(2,2), padding='same'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001),
                     loss='categorical_crossentropy',
                     metrics = ['acc']
                     )
history = model.fit(train_data, train_label,
                        validation_data=(val_data, val_label),
                        epochs=20,
                        batch_size=64
                        )

Epoch 1/20
199/199 [==============================] - 4s 11ms/step - loss: 0.5561 - acc: 0.8196 - val_loss: 0.4723 - val_acc: 0.8648
Epoch 2/20
199/199 [==============================] - 2s 10ms/step - loss: 0.1817 - acc: 0.9400 - val_loss: 0.2413 - val_acc: 0.9255
Epoch 3/20
199/199 [==============================] - 3s 13ms/step - loss: 0.1241 - acc: 0.9582 - val_loss: 0.2883 - val_acc: 0.9267
Epoch 4/20
199/199 [==============================] - 2s 12ms/step - loss: 0.0988 - acc: 0.9657 - val_loss: 0.2139 - val_acc: 0.9409
Epoch 5/20
199/199 [==============================] - 2s 10ms/step - loss: 0.0706 - acc: 0.9775 - val_loss: 0.2133 - val_acc: 0.9418
Epoch 6/20
199/199 [==============================] - 2s 10ms/step - loss: 0.0591 - acc: 0.9800 - val_loss: 0.2874 - val_acc: 0.9252
Epoch 7/20
199/199 [==============================] - 2s 10ms/step - loss: 0.0542 - acc: 0.9816 - val_loss: 0.1330 - val_acc: 0.9632
Epoch 8/20
199/199 [==============================] - 2s 10ms/step - 

### Testing the model

In [12]:

# Make predictions on the test set
predictions = model.predict(test_data)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == np.argmax(test_label, axis=1))

print(f"Test Accuracy: {accuracy:.4f}")

125/125 [==============================] - 0s 2ms/step
Test Accuracy: 0.9532
